1.0 PRÁCTICA

1.1 Extracción de datos

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# CSV ya arreglado: separado por comas
data = pd.read_csv('data/drugsComTest_raw.csv')

data.head()

data.info()

<class 'pandas.DataFrame'>
RangeIndex: 53766 entries, 0 to 53765
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   53766 non-null  int64  
 1   drugName     53766 non-null  str    
 2   condition    53471 non-null  str    
 3   review       53766 non-null  str    
 4   rating       53766 non-null  float64
 5   date         53766 non-null  str    
 6   usefulCount  53766 non-null  int64  
dtypes: float64(1), int64(2), str(4)
memory usage: 2.9 MB


1.2 Trato de datos NULL

In [17]:
#Tabla rating
# 0) Comprobación rápida antes de limpiar
print("dtype rating (antes):", data["rating"].dtype)
print("nulos rating (antes):", data["rating"].isna().sum())

# 1) Forzar conversión a numérico (lo no-convertible pasa a NaN)
data["rating"] = pd.to_numeric(data["rating"], errors="coerce")

# 2) Contar los NaN (después de convertir)
print(f"La suma de valores nulos en rating es: {data['rating'].isna().sum()}")

# 3) Check de rango (ratings deberían estar entre 0 y 10)
print("min rating:", data["rating"].min(), "max rating:", data["rating"].max())

print(f"La mediana de los ratings es: {data['rating'].median()}")
#Como rating no es nuestra variavle explicita y no sesgará el resultado la vamos a modificar
#Utilizaremos la mediana para rellenar los valores nulos 
data["rating"] = data["rating"].fillna(data["rating"].median())



La suma de valores nulos en rating es: 0
La mediana de los ratings es: 8.0


In [ ]:
# Tabla drugName
# 0) Comprobación rápida antes de limpiar
print("dtype drugName (antes):", data["drugName"].dtype)
print("nulos drugName (antes):", data["drugName"].isna().sum())

# 1) Asegurar tipo string
data["drugName"] = data["drugName"].astype(str)
data["drugName"] = data["drugName"].replace("", np.nan).replace("nan", np.nan)

# 2) Contar los NaN (después de limpiar)
print(f"La suma de valores nulos en drugName es: {data['drugName'].isna().sum()}")

# 3) Moda para rellenar nulos
print(f"La moda de drugName es: {data['drugName'].mode().iloc[0]}")
data["drugName"] = data["drugName"].fillna(data["drugName"].mode().iloc[0])

In [ ]:
# Tabla condition
# 0) Comprobación rápida antes de limpiar
print("dtype condition (antes):", data["condition"].dtype)
print("nulos condition (antes):", data["condition"].isna().sum())

# 1) Asegurar tipo string
data["condition"] = data["condition"].astype(str)
data["condition"] = data["condition"].replace("", np.nan).replace("nan", np.nan)

# 2) Contar los NaN (después de limpiar)
print(f"La suma de valores nulos en condition es: {data['condition'].isna().sum()}")

# 3) Moda para rellenar nulos
print(f"La moda de condition es: {data['condition'].mode().iloc[0]}")
data["condition"] = data["condition"].fillna(data["condition"].mode().iloc[0])

In [ ]:
# Tabla review
# 0) Comprobación rápida antes de limpiar
print("dtype review (antes):", data["review"].dtype)
print("nulos review (antes):", data["review"].isna().sum())

# 1) Asegurar tipo string
data["review"] = data["review"].astype(str)
data["review"] = data["review"].replace("", np.nan).replace("nan", np.nan)

# 2) Contar los NaN (después de limpiar)
print(f"La suma de valores nulos en review es: {data['review'].isna().sum()}")

# 3) Rellenar nulos con placeholder (la moda en textos largos puede no ser representativa)
data["review"] = data["review"].fillna("(No review)")

In [ ]:
# Tabla date
# 0) Comprobación rápida antes de limpiar
print("dtype date (antes):", data["date"].dtype)
print("nulos date (antes):", data["date"].isna().sum())

# 1) Convertir a datetime (lo no convertible pasa a NaT)
data["date"] = pd.to_datetime(data["date"], errors="coerce")

# 2) Contar los NaN (después de convertir)
print(f"La suma de valores nulos en date es: {data['date'].isna().sum()}")

# 3) Rellenar nulos con la moda (fecha más frecuente)
print(f"La moda de date es: {data['date'].mode().iloc[0]}")
data["date"] = data["date"].fillna(data["date"].mode().iloc[0])

In [ ]:
# Tabla usefulCount
# 0) Comprobación rápida antes de limpiar
print("dtype usefulCount (antes):", data["usefulCount"].dtype)
print("nulos usefulCount (antes):", data["usefulCount"].isna().sum())

# 1) Forzar conversión a numérico (lo no-convertible pasa a NaN)
data["usefulCount"] = pd.to_numeric(data["usefulCount"], errors="coerce")

# 2) Contar los NaN (después de convertir)
print(f"La suma de valores nulos en usefulCount es: {data['usefulCount'].isna().sum()}")

# 3) Check de rango (conteos no negativos)
print("min usefulCount:", data["usefulCount"].min(), "max usefulCount:", data["usefulCount"].max())

print(f"La mediana de usefulCount es: {data['usefulCount'].median()}")
data["usefulCount"] = data["usefulCount"].fillna(data["usefulCount"].median())